# Projet E4

## Introduction

Modèle IA pour la reconnaissance d'anomalie chez le patient.
* Input : fichier .wav d'un cardiogramme
* Process : wav -> spectrogram/png -> neural convolution network
* Output : le type d'anomalie 

## Import

In [1]:
import wave
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import signal
from scipy.io import wavfile
import splitfolders
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, add
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

import cv2

# Traitement des données .wav

Les fonctions ci-dessous ont pour rôle dans un premier temps de convertir les fichiers .wav contenu dans Data/MLDataset en spectrogramme au format .png dans un dossier Data/Spectrograms/(Known_datas/Unknown_datas)/(Catégories).

* Known_datas : le dossier où sont contenu les datas dont on connait le label (serviront de training et testing set).
* Unknown_datas : le dossier contenant des datas dont on ne connait pas le label.

In [37]:
# Récupère un fichier dont le chemin d'accès est path.
# On utilise scipy.io.wavfile.read pour récupérer la fréquence (sample_rate) et les datas (sample) du fichier .wav. 
# On s'assure de créer le dossier où l'on veut stocker les png.
# On monte le spectrogramme à partir de sample_rate et samples puis on sauvegarde sans les axes au format png. 
def wave_to_spect(path):
    path_tokens = path.split("/")
    sample_rate, samples = wavfile.read(path)
    os.makedirs("Data/Spectrograms/"+path_tokens[2]+"/"+path_tokens[3],exist_ok = True)
    plt.subplot(212)
    plt.specgram(samples, Fs=sample_rate,NFFT=2048,Fc = 0, sides='default',mode='default', scale='dB')
    plt.axis('off')
    plt.savefig("Data/Spectrograms/"+path_tokens[2]+"/"+path_tokens[3]+"/"+path_tokens[4].replace(".wav",".png"))
    plt.clf()




In [48]:


#Utilisation de la fonction décrite plus haut sur l'ensemble des données. 
data_path = "Data/MLDataset"
for t in os.listdir(data_path):
    complete_data_path = os.path.join(data_path,t)
    for a in os.listdir(complete_data_path ):
        new_data_path = os.path.join(complete_data_path,a)
        for filename in os.listdir(new_data_path):
            wave_to_spect(os.path.join(new_data_path,filename).replace("\\","/"))

<Figure size 432x288 with 0 Axes>

In [53]:
#Split des données aux labels connus pour créer un train_set et un validation_set
splitfolders.ratio('./Data/Spectrograms/Known_datas',output="./Data/Spectrograms/Training",seed=1337,ratio=(.8,.2))

Copying files: 517 files [00:01, 375.56 files/s]


# Modèle CovNet

Création des train_set,test_set et du modèle.

In [122]:
#Création du train_set et du test_set. Rescale des valeurs de couleurs de 0 à 255 vers une normalisation de 0 à 1.

train_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    './Data/Spectrograms/Training/train',
    target_size = (288, 432),
    batch_size= 19,
    class_mode = 'categorical',
    shuffle=False
)

test_set = test_datagen.flow_from_directory(
    './Data/Spectrograms/Training/val',
    target_size = (288, 432),
    batch_size= 19,
    class_mode = 'categorical',
    shuffle=False
)

Found 412 images belonging to 5 classes.
Found 105 images belonging to 5 classes.


In [114]:
# Création du modèle et ajout des layers.
# Les modifications des weights et sur les layers n'ont pas encore été efféctué.

model = Sequential()
input_shape = (288, 432, 3)#first hidden layer
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(AveragePooling2D((2,2)))
model.add(Activation('relu'))#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))#3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.2))#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))#Output layer
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 286, 430, 32)      896       
                                                                 
 average_pooling2d_20 (Avera  (None, 143, 215, 32)     0         
 gePooling2D)                                                    
                                                                 
 activation_30 (Activation)  (None, 143, 215, 32)      0         
                                                                 
 conv2d_21 (Conv2D)          (None, 143, 215, 64)      18496     
                                                                 
 average_pooling2d_21 (Avera  (None, 71, 107, 64)      0         
 gePooling2D)                                                    
                                                                 
 activation_31 (Activation)  (None, 71, 107, 64)      

In [115]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [116]:
#Entrainement du modèle
model.fit_generator(
        training_set,
        steps_per_epoch=4,
        epochs=50,
        validation_data=test_set,
        validation_steps=200)

C:\Users\pc\AppData\Local\Temp/ipykernel_31648/2132631962.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
4/4 [==============================] - 9s 2s/step - loss: 1.5892 - accuracy: 0.2857 - val_loss: 1.5121 - val_accuracy: 0.6095
Epoch 2/50
4/4 [==============================] - 5s 1s/step - loss: 1.5499 - accuracy: 0.1184
Epoch 3/50
4/4 [==============================] - 6s 1s/step - loss: 1.2689 - accuracy: 0.6579
Epoch 4/50
4/4 [==============================] - 5s 1s/step - loss: 0.3154 - accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 6s 1s/step - loss: 2.0149 - accuracy: 0.2895
Epoch 6/50
4/4 [==============================] - 5s 1s/step - loss: 1.1775 - accuracy: 0.4571
Epoch 7/50
4/4 [==============================] - 6s 1s/step - loss: 1.4384 - accuracy: 0.3421
Epoch 8/50
4/4 [==============================] - 5s 1s/step - loss: 0.8383 - accuracy: 0.5857
Epoch 9/50
4/4 [==============================] - 5s 1s/step - loss: 1.9743 - accuracy: 0.5000
Epoch 10/50
4/4 [==============================] - 5s 1s/step - loss: 1.0763 - accuracy: 1.0000
Epoch 1

In [117]:
#Evaluation du modèle
model.evaluate_generator(generator = test_set, steps=4)

C:\Users\pc\AppData\Local\Temp/ipykernel_31648/2186847382.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator = test_set, steps=4)


[1.6792891025543213, 0.46052631735801697]

In [118]:
#Prediction sur les données test
test_set.reset()
pred = model.predict_generator(test_set, steps=13, verbose=1)

C:\Users\pc\AppData\Local\Temp/ipykernel_31648/2506516680.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_set, steps=13, verbose=1)


13/13 [==============================] - 3s 175ms/step


In [119]:
# Labélisation des résultats
predicted_class_indices = np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames = test_set.filenames

{0: 'Artifact', 1: 'Extrahs', 2: 'Extrasystole', 3: 'Murmur', 4: 'Normal'}


In [120]:
#On vérifie que le nombre de prediction = le nombre de files
print(len(filenames),len(predictions))

105 105


In [121]:
#Création d'un csv pour visualiser les résultats.
results = pd.DataFrame({"Filename":filenames,
"Predictions":predictions})
results.to_csv('prediction_results.csv',index=False)